In [1]:
import os
from spark_dbconnect import spark_start, spark_stop

In [3]:
config = {
    "spark.databricks.service.server.enabled": True,
    "spark.databricks.service.address": "https://adb-2740165739887726.6.azuredatabricks.net",
    "spark.databricks.service.token": os.environ["DATABRICKS_TOKEN"],
    "spark.databricks.service.clusterId": "0802-221206-sols809",
    "spark.databricks.service.orgId": "2740165739887726",
    "spark.databricks.service.port": "15001"
}
spark = spark_start(config=config)

In [4]:
df = spark.read.parquet("dbfs:/FileStore/tables/BroadcastXY.parq")
df.createOrReplaceTempView("v0")

In [5]:
df.printSchema()

root
 |-- OBJECTID: long (nullable = true)
 |-- SOG: long (nullable = true)
 |-- COG: long (nullable = true)
 |-- Heading: long (nullable = true)
 |-- ROT: long (nullable = true)
 |-- BaseDateTime: timestamp (nullable = true)
 |-- Status: long (nullable = true)
 |-- VoyageID: long (nullable = true)
 |-- MMSI: long (nullable = true)
 |-- ReceiverType: string (nullable = true)
 |-- ReceiverID: string (nullable = true)
 |-- Shape@X: double (nullable = true)
 |-- Shape@Y: double (nullable = true)



In [6]:
cell0 = 100.0 # meters
cell1 = cell0 * 0.5

rows = spark\
    .sql(f"""
select q*{cell0}+{cell1} x,r*{cell0}+{cell1} y,least(count(1),1000) as pop
from
(select cast(`Shape@X`/{cell0} as long) q,cast(`Shape@Y`/{cell0} as long) r from v0)
group by q,r
""")\
    .collect()

In [7]:
ws = "memory"
nm = "Bins"

fc = os.path.join(ws,nm)

arcpy.management.Delete(fc)

sp_ref = arcpy.SpatialReference(3857)
arcpy.management.CreateFeatureclass(ws,nm,"POINT",spatial_reference=sp_ref)
arcpy.management.AddField(fc, "POP", "LONG")

with arcpy.da.InsertCursor(fc, ["SHAPE@X","SHAPE@Y", "POP"]) as cursor:
    for row in rows:
        cursor.insertRow(row)

In [8]:
spark_stop()